# Test functions in matrix_ntru_system.sage

- Test basic encryption and decryption
- Test nuances for the proposed attack in 
"2024, SBSEG. Lattice Base Reduction Attack on Matrix NTRU. Thiago Sousa and Tertuliano Carneiro"
- Analyses conditions for decryption to work (it is a probabilistic system and it can fail sometimes)

In [ ]:
load("matrix_ntru_system.sage")
def print2(*args):
    for arg in args:
        print('-----------')
        print(arg)

In [ ]:
# setParameters and getParameters
setParameters([13,33])
print(n,p,q,Rp,Rq)
getParameters()

setParameters([32,13])
print(n,p,q,Rp,Rq)
getParameters()

In [ ]:
# keygen testing if inverses were found for X and if H satisfy X * H = p * Y
load("matrix_ntru_system.sage")
rep = 10
nvals = range(2,100,5)
qvals = [2^i for i in range(1, 10+1)]

for n in nvals:
    #print(n)
    for q in qvals:
        setParameters([n,q]) # parameters n and q.
        X,Y,Xp,Xq,H = keygen()
        XXp = X * Xp # linear algebra result: when Xp exists, then X commutes with Xp. 
        XpX = Xp * X
        XXq = X * Xq
        XqX = Xq * X
        cond1 = (XXp == XpX) and (XXp == identity_matrix(n))
        cond2 = (XXq == XqX) and (XXq == identity_matrix(n))
        cond3 = X * H == p * Y
        if( not(cond1 and cond2 and cond3) ):
            print("keygen test failed")
            break
print("keygen test finished sucessfully!")
    

In [ ]:
# encrypt and decrypt test. test for a fixed value of n = 4 and q = 32
# that is the case of the example from Nayak. 
load("matrix_ntru_system.sage")
setParameters([3,32]) # parameters n and q.
getParameters()
rep = pow(10,3)
failed = 0
for i in range(rep):
    X,Y,Xp,Xq,H = keygen()
    M = randomMessage()
    E = encrypt(M,H)
    C = decrypt(E,X,Xp)
    if(C != M):
        failed += 1
print(failed)

In [ ]:
# Generate a table with decryption failure for different n,q values
# conjecture according to silvermans proof of p. 408, 
#  q > 2n(p+1) ensures 0 decryption failure probability. 
results = list()
nvals = range(4,50,4)
qvals = [10,30,79,115,116,117,118,119,120,121,122,123] #[14, 16, 17, 19] #[ 29, 31, 32, 34, 35]
#[2^i for i in range(2, 10+1)]# + [107, 211, 307,401, 503,  601, 701, 809, 907, 1009] # powers of 2 till 1024 and the first prime following 100,200,...till 1000
qvals.sort()
total = 10000


for n in nvals:
    for q in qvals:
        setParameters([n,q]) # parameters n and q.
        decryptionWorks = (q > 2*n*(p+1))
        failed = 0
        for i in range(total):
            X,Y,Xp,Xq,H = keygen()
            M = randomMessage()
            E = encrypt(M,H)
            C = decrypt(E,X,Xp)
            if(C != M):
                failed += 1
        results.append([n,q,total,failed,decryptionWorks])
results

In [ ]:
# decryption failure investigation for fixed n and growing q
qlist = [ 16, 17, 19, 20, 22, 23, 25, 26, 28, 29, 31, 32, 34, 35, 37, 38, 40, 41, 43, 44, 46, 47, 49, 50, 52, 53, 55, 56, 58, 59, 61, 62, 64]


                   q > 2 * n * ( p + 1)
n,p = 4,3, we need q > 2*4*(3 + 1) = 32. 


n,p = 8,3, we need q > 2*8*(3 + 1) = 64. 
n,p = 15,3, we need q > 2*15*(3 + 1) = 120. So try q = 121. 


# Investigate if finding a permutation of X, allows us to decrypt correctly 


In [ ]:
from sage.combinat.permutation import Permutations

# Generate a random permutation of range(10)
def Xperm(X):
    # given X, returns a random permutation of lines of X, where 
    # each line is also randomly multiplied by 1 or -1. 
    Xdim = X.dimensions()[0]
    a = Permutations(Xdim).random_element()
    b = [i - 1 for i in a]
    X2 = random_matrix(ZZ, nrows = Xdim, ncols = Xdim, x=-1, y=2)
    for i in range(Xdim):
        sign_line = 2 * (randrange(2) > 0) - 1 # random in {-1,1}
        X2[i,:] = sign_line * X[b[i],:]
    return(X2)

In [ ]:
setParameters([3,32])
X,Y,Xp,Xq,H = keygen()
print2(X,Xperm(X))

In [ ]:
load("matrix_ntru_system.sage")
setParameters([50,1024])
X,Y,Xp,Xq,H = keygen()
M = randomMessage()
E = encrypt(M,H)
#C = decrypt(E,X,Xp)
#C == M
X2 = Xperm(X)
X2p = Matrix(Rp,X2).inverse()
C2 = decrypt(E,X2,X2p)
#print(X != X2)
#print2(X,X2)
#print2(C2,M)
C2 == M